In [86]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import cpca
import os
os.chdir("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据")


In [84]:
df = pd.read_csv("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据/RFA_protest3.csv",encoding='utf-8')
df = df[['adcode','location','size_level','year','month','day','citycode','省','市','区']]
df.to_csv("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据/RFA_protest3_cropped.csv",index=False)

In [87]:
df = pd.read_stata("cloudseeding.dta")
df["district"] = df['prov'] + df['city'] + df['county']

In [88]:
df_adcode = cpca.transform(df["district"])
df['adcode'] = df_adcode['adcode']
df.to_csv("cloudseeding_adcode.csv")